In [1]:
import pandas as pd

In [7]:
rating = pd.read_csv("datasets/score.csv")
titles = pd.read_csv("datasets/titles.csv")

In [8]:
#quitar duplicados
rating = rating.drop_duplicates(['userId','movieId'],keep='last')
# se realiza modificaciones y se elimina los datos que no son importantes
titles = titles.drop(['rating','show_id','type','director','cast','country','date_added','release_year','duration','description','platform','duration_int','duration_type'],1)
rating = rating.drop('timestamp',1)
titles.rename(columns={'id': 'movieId'}, inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_6928\4158490137.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  titles = titles.drop(['rating','show_id','type','director','cast','country','date_added','release_year','duration','description','platform','duration_int','duration_type'],1)
C:\Users\PC\AppData\Local\Temp\ipykernel_6928\4158490137.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  rating = rating.drop('timestamp',1)


In [9]:
userId= 1
movieId = 'as2289'

# se realiza la agregacion de los generos en columnas nuevas individualmente a una copia del dataframe titles
titles_copy = titles.copy()
for index, row in titles.iterrows():
    for genre in row['listed_in']:
        titles_copy.at[index, genre] = 1
# se asigna 0 a los campos nulos
titles_copy = titles_copy.fillna(0)
# se obtiene las peliculas calificadas por el usuario
user_rating_title = rating[rating['userId']==userId]
# se asigna los generos con 0 y 1 a las peliculas califidadas por el usuario 
title_user = titles_copy[titles_copy['movieId'].isin(user_rating_title['movieId'].tolist())]
# limpiar informacion innecesaria
title_user = title_user.reset_index(drop = True)
table_genres = title_user.drop(['movieId','title','listed_in'], 1)
# se crea la matriz que refleja las preferencias del usuario en funcion al genero
title_user = table_genres.transpose().dot(user_rating_title['rating'])
# se extrae los generos de la tabla original
genres = titles_copy.set_index(titles_copy['movieId'])
# se borra informacion innecesaria
genres = genres.drop(['movieId','title','listed_in'], 1)
genres.shape
# se calcula el promedio ponderado de las peliculas relacionadas con las peliculas de calificadas por el usuario
recom = ((genres * title_user).sum(axis=1))/(title_user.sum())

# se valida la pelicula dada por el usuario para verificar si tiene un promedio ponderado superior a cero
result=''
if(recom.filter(like=movieId).values[0] > 0):
    result = "recomendado"
else:
    result = "no recomendado"

result

C:\Users\PC\AppData\Local\Temp\ipykernel_6928\1504229114.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table_genres = title_user.drop(['movieId','title','listed_in'], 1)
C:\Users\PC\AppData\Local\Temp\ipykernel_6928\1504229114.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genres = genres.drop(['movieId','title','listed_in'], 1)


'recomendado'